<a href="https://colab.research.google.com/github/rajanaids-hub/Reinforcement_Learning_Lab/blob/main/TD_Learning_(SARSA)_Exp8v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML, display
import random

# =====================================================================
# 🎓 STUDENT EXPERIMENT SETTINGS 🎓
# =====================================================================
GRID_SIZE = 4           # Try 5 or 6 (Size of the grid)
EPISODES = 500          # Try 100 or 1000 (How many times the agent plays)
ALPHA = 0.1             # Learning Rate (0 to 1): How fast it overrides old info
GAMMA = 0.9             # Discount Factor (0 to 1): Importance of future rewards
EPSILON = 0.2           # Exploration Rate: 20% chance to pick a random path
STEP_REWARD = -1.0      # Penalty for each step taken
# =====================================================================

# 1. Define the Environment (Interactive for TD Learning)
class GridworldEnv:
    def __init__(self, size=GRID_SIZE):
        self.size = size
        self.nS = size * size
        self.nA = 4 # 0: UP, 1: RIGHT, 2: DOWN, 3: LEFT
        self.start_states = list(range(1, self.nS - 1)) # Exclude goal states

    def reset(self):
        # Start at a random non-goal state for better exploration
        self.state = random.choice(self.start_states)
        return self.state

    def step(self, action):
        row, col = divmod(self.state, self.size)

        if action == 0: row = max(row - 1, 0)               # UP
        elif action == 1: col = min(col + 1, self.size - 1) # RIGHT
        elif action == 2: row = min(row + 1, self.size - 1) # DOWN
        elif action == 3: col = max(col - 1, 0)             # LEFT

        self.state = (row * self.size) + col
        done = (self.state == 0 or self.state == self.nS - 1) # Reached corners?
        reward = 0.0 if done else STEP_REWARD

        return self.state, reward, done

# 2. TD Learning Agent (SARSA)
class SarsaAgent:
    def __init__(self, nS, nA, alpha, gamma, epsilon):
        self.nA = nA
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        # Initialize Q-Table to zeros
        self.q_table = np.zeros((nS, nA))

    def choose_action(self, state):
        """ Epsilon-Greedy Policy """
        if random.uniform(0, 1) < self.epsilon:
            return random.randint(0, self.nA - 1) # Explore
        else:
            # Exploit best action (break ties randomly)
            max_q = np.max(self.q_table[state])
            best_actions = np.where(self.q_table[state] == max_q)[0]
            return random.choice(best_actions)

    def learn(self, state, action, reward, next_state, next_action):
        """ The SARSA Update Rule """
        current_q = self.q_table[state, action]
        next_q = self.q_table[next_state, next_action]

        # 🧠 Temporal Difference Target
        td_target = reward + self.gamma * next_q
        td_error = td_target - current_q

        # Update Q-value
        self.q_table[state, action] = current_q + self.alpha * td_error

# 3. Helper to extract V(s) and Policy from Q-Table for visualization
def extract_v_and_policy(q_table, nS, nA):
    V = np.max(q_table, axis=1) # Value is the max Q-value for that state
    policy = np.zeros((nS, nA))
    for s in range(nS):
        best_a = np.argmax(q_table[s])
        policy[s][best_a] = 1.0 # Greedy policy
    return V, policy

# =====================================================================
# 📉 VISUALIZATION CODE (Matches previous interactive style)
# =====================================================================
def animate_learning(history, size):
    fig, ax = plt.subplots(figsize=(6, 6))
    arrows = {0: '↑', 1: '→', 2: '↓', 3: '←'}

    def update(frame):
        ax.clear()
        # Ensure we don't go out of bounds on the last frame
        actual_frame = min(frame * 10, len(history) - 1)
        V, policy, episode = history[actual_frame]

        ax.matshow(V.reshape(size, size), cmap='Blues')

        for s in range(len(V)):
            row, col = divmod(s, size)
            ax.text(col, row - 0.2, f"{V[s]:.1f}", va='center', ha='center', color='black')

            if s != 0 and s != len(V) - 1:
                best_act = np.argmax(policy[s])
                ax.text(col, row + 0.2, arrows[best_act], va='center', ha='center', color='red', fontsize=16, weight='bold')
            else:
                ax.text(col, row + 0.2, "GOAL", va='center', ha='center', color='green', fontsize=10, weight='bold')

        ax.set_xticks([]); ax.set_yticks([])
        ax.set_title(f"TD Learning (SARSA) - Episode: {episode}", pad=20)

    # We take snapshots, so we calculate total frames based on history length divided by snapshot interval
    total_frames = max(1, len(history) // 10)
    anim = animation.FuncAnimation(fig, update, frames=total_frames, interval=200, repeat=False)
    plt.close(fig)
    display(HTML(anim.to_jshtml()))

# =====================================================================
# 🚀 MAIN EXECUTION LOOP
# =====================================================================
if __name__ == "__main__":
    env = GridworldEnv(size=GRID_SIZE)
    agent = SarsaAgent(nS=env.nS, nA=env.nA, alpha=ALPHA, gamma=GAMMA, epsilon=EPSILON)

    history = [] # Store learning progress

    print(f"Starting TD Learning (SARSA) for {EPISODES} episodes...")

    for episode in range(1, EPISODES + 1):
        state = env.reset()
        action = agent.choose_action(state) # Choose A from S

        done = False
        while not done:
            # Take action A, observe R, S'
            next_state, reward, done = env.step(action)

            # Choose A' from S' (This is what makes it SARSA and not Q-learning)
            next_action = agent.choose_action(next_state)

            # Agent updates its Q-Table
            agent.learn(state, action, reward, next_state, next_action)

            state = next_state
            action = next_action

        # Record a snapshot of the agent's brain every episode for animation
        V, policy = extract_v_and_policy(agent.q_table, env.nS, env.nA)
        history.append((V.copy(), policy.copy(), episode))

        if episode % 100 == 0:
            print(f"Completed Episode {episode}")

    print("🎉 Training complete! Rendering animation...")
    animate_learning(history, GRID_SIZE)

Starting TD Learning (SARSA) for 500 episodes...
Completed Episode 100
Completed Episode 200
Completed Episode 300
Completed Episode 400
Completed Episode 500
🎉 Training complete! Rendering animation...
